In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy

np.random.seed(42)
tf.random.set_seed(42)


def build_encoder(input_dim, latent_dim):
    inputs = Input(shape=(input_dim,))
    h = Dense(256, activation='relu')(inputs)
    z_mean = Dense(latent_dim)(h)
    z_log_var = Dense(latent_dim)(h)
    return Model(inputs, [z_mean, z_log_var], name='encoder')

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_decoder(latent_dim, original_dim):
    latent_inputs = Input(shape=(latent_dim,))
    h = Dense(256, activation='relu')(latent_inputs)
    outputs = Dense(original_dim, activation='sigmoid')(h)
    return Model(latent_inputs, outputs, name='decoder')

def build_vae(input_dim, latent_dim):
    encoder = build_encoder(input_dim, latent_dim)
    decoder = build_decoder(latent_dim, input_dim)

    inputs = Input(shape=(input_dim,))
    z_mean, z_log_var = encoder(inputs)
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
    outputs = decoder(z)

    vae = Model(inputs, outputs, name='vae')

    # Define the loss function
    xent_loss = input_dim * binary_crossentropy(inputs, outputs)
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)

    vae.add_loss(vae_loss)

    return vae, encoder, decoder

# Define data dimensions and create VAE
input_dim = 784  # Adjust based on your input data dimension
latent_dim = 2   # Adjust based on desired latent space dimension
vae, encoder, decoder = build_vae(input_dim, latent_dim)

# Compile the VAE
vae.compile(optimizer='adam')

2023-11-09 18:21:53.048226: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 18:21:53.077819: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 18:21:53.077853: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 18:21:53.077876: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 18:21:53.083797: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 18:21:53.084447: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens